# CH11 Training Deep Neural Networks


In [1]:
import tensorflow as tf

In [2]:
tf.__version__

'2.10.1'

### Activations and he initializations


In [3]:
dense = tf.keras.layers.Dense(
    50, activation='relu', kernel_initializer='he_normal')

In [9]:
dense.kernel_initializer.distribution

'truncated_normal'

In [10]:
he_avg_init = tf.keras.initializers.VarianceScaling(scale=2., mode="fan_avg",
                                                    distribution="uniform")
dense = tf.keras.layers.Dense(50, activation="sigmoid",
                              kernel_initializer=he_avg_init)

leaky relu


In [11]:
leaky_relu = tf.keras.layers.LeakyReLU(alpha=.2)
dense = tf.keras.layers.Dense(
    50, activation=leaky_relu, kernel_initializer='he_normal')

elu


In [12]:
dense = tf.keras.layers.Dense(
    50, activation='elu', kernel_initializer='he_normal')

`SELU` , but it has few considerations:


• The input features must be standardized: mean 0 and standard deviation 1.

• Every hidden layer’s weights must be initialized using LeCun normal initializa
tion. In Keras, this means setting kernel_initializer="lecun_normal".

• The self-normalizing property is only guaranteed with plain MLPs.

• You cannot use regularization techniques like ℓ1
or ℓ2
regularization, max-norm,
batch-norm, or regular dropout.


In [13]:
dense = tf.keras.layers.Dense(
    50, activation='selu', kernel_initializer='lecun_normal')

an example of a self-regularized network using SELU


In [14]:
tf.random.set_seed(42)
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))
for i in range(100):
    model.add(tf.keras.layers.Dense(100, activation='selu',
              kernel_initializer='lecun_normal'))
model.add(tf.keras.layers.Dense(10, activation='softmax'))

In [3]:
model.compile(loss='sparse_categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(
    learning_rate=0.001), metrics=['accuracy'])


NameError: name 'model' is not defined

In [4]:

fashion_mnist = tf.keras.datasets.fashion_mnist.load_data()
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist
X_train, y_train = X_train_full[:-5000], y_train_full[:-5000]
X_valid, y_valid = X_train_full[-5000:], y_train_full[-5000:]
X_train, X_valid, X_test = X_train / 255, X_valid / 255, X_test / 255

class_names = ["T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
               "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot"]

In [5]:
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means)/pixel_stds
X_valid_scaled = (X_valid - pixel_means)/pixel_stds
X_test_scaled = (X_test - pixel_means)/pixel_stds

In [17]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 36s 18ms/step - loss: 1.0050 - accuracy: 0.6275 - val_loss: 0.7212 - val_accuracy: 0.7350
Epoch 2/5
1719/1719 [==============================] - 33s 19ms/step - loss: 0.6421 - accuracy: 0.7656 - val_loss: 0.5779 - val_accuracy: 0.7892
Epoch 3/5
1719/1719 [==============================] - 32s 19ms/step - loss: 0.5580 - accuracy: 0.7990 - val_loss: 0.5336 - val_accuracy: 0.8102
Epoch 4/5
1719/1719 [==============================] - 30s 18ms/step - loss: 0.5159 - accuracy: 0.8169 - val_loss: 0.4961 - val_accuracy: 0.8254
Epoch 5/5
1719/1719 [==============================] - 33s 19ms/step - loss: 0.4952 - accuracy: 0.8280 - val_loss: 0.4968 - val_accuracy: 0.8308


The network managed to learn, despite how deep it is. Now look at what happens if we try to use the ReLU activation function instead:


In [18]:
model = tf.keras.Sequential()
model.add(tf.keras.layers.Flatten(input_shape=[28, 28]))
for layer in range(100):
    model.add(tf.keras.layers.Dense(100, activation="relu",
                                    kernel_initializer="he_normal"))
model.add(tf.keras.layers.Dense(10, activation="softmax"))

In [19]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
              metrics=["accuracy"])

In [20]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 34s 18ms/step - loss: 1.9997 - accuracy: 0.2147 - val_loss: 1.7337 - val_accuracy: 0.3048
Epoch 2/5
1719/1719 [==============================] - 30s 17ms/step - loss: 1.2544 - accuracy: 0.4702 - val_loss: 1.3963 - val_accuracy: 0.3952
Epoch 3/5
1719/1719 [==============================] - 31s 18ms/step - loss: 1.0043 - accuracy: 0.5888 - val_loss: 1.6453 - val_accuracy: 0.3570
Epoch 4/5
1719/1719 [==============================] - 31s 18ms/step - loss: 0.9494 - accuracy: 0.6116 - val_loss: 0.7680 - val_accuracy: 0.7068
Epoch 5/5
1719/1719 [==============================] - 31s 18ms/step - loss: 0.8271 - accuracy: 0.6733 - val_loss: 0.7861 - val_accuracy: 0.6828


Not great at all, we suffered from the vanishing/exploding gradients problem.


In [ ]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

GELU, Swish and Mish


In [22]:
def mish(x):
    return x * tf.math.tanh(tf.math.softplus(x))


# Using GELU activation
dense_gelu = tf.keras.layers.Dense(
    50, activation=tf.keras.activations.gelu, kernel_initializer='he_normal')

# Using built-in Swish activation
dense_swish = tf.keras.layers.Dense(
    50, activation='swish', kernel_initializer='he_normal')

# Using Mish activation
dense_mish = tf.keras.layers.Dense(
    50, activation=mish, kernel_initializer='he_normal')

---


### Batch Normalization


In [23]:
tf.keras.backend.clear_session()
tf.random.set_seed(42)

In [26]:
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization

model = tf.keras.Sequential([
    Flatten(input_shape=[28, 28]),
    BatchNormalization(),
    Dense(300, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dense(100, activation='relu', kernel_initializer='he_normal'),
    BatchNormalization(),
    Dense(10, activation='softmax'),

])

In [27]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_1 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [33]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [34]:
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
              metrics="accuracy")
model.fit(X_train, y_train, epochs=2, validation_data=(X_valid, y_valid))

Epoch 1/2
1719/1719 [==============================] - 10s 5ms/step - loss: 0.5548 - accuracy: 0.8077 - val_loss: 0.3976 - val_accuracy: 0.8580
Epoch 2/2
1719/1719 [==============================] - 9s 5ms/step - loss: 0.4069 - accuracy: 0.8550 - val_loss: 0.3608 - val_accuracy: 0.8686


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a BatchNormalization layer does not need to have bias terms, since the BatchNormalization layer some as well, it would be a waste of parameters, so you can set use_bias=False when creating those layers:


In [35]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Activation
tf.keras.backend.clear_session()
tf.random.set_seed(42)


model = Sequential([
    Flatten(input_shape=[28, 28]),
    Dense(300, kernel_initializer='he_normal'),
    BatchNormalization(),
    Activation('relu'),
    Dense(100, kernel_initializer='he_normal'),
    BatchNormalization(),
    Activation('relu'),
    Dense(10),
    Activation('softmax')
])

model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd",
              metrics="accuracy")
model.fit(X_train, y_train, epochs=1, validation_data=(X_valid, y_valid))

1719/1719 [==============================] - 9s 5ms/step - loss: 0.6099 - accuracy: 0.7979 - val_loss: 0.4347 - val_accuracy: 0.8442


### Gradient Clipping

to avoid gradients explosion


All tf.keras.optimizers accept clipnorm or clipvalue arguments:


In [37]:
optim = tf.keras.optimizers.SGD(clipvalue=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optim)

In [38]:
optim = tf.keras.optimizers.SGD(clipnorm=1.0)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optim)

---


### Reusing Pre-trained Layers

In [7]:
import numpy as np

split Fashion MNIST into tasks A and B, then train and save model A to "my_model_A".

In [8]:
pos_class_id = class_names.index("Pullover")
neg_class_id = class_names.index("T-shirt/top")


def split_dataset(X, y):
    y_for_B = (y == pos_class_id) | (y == neg_class_id)
    y_A = y[~y_for_B]
    y_B = (y[y_for_B] == pos_class_id).astype(np.float32)
    old_class_ids = list(set(range(10)) - set([neg_class_id, pos_class_id]))
    for old_class_id, new_class_id in zip(old_class_ids, range(8)):
        y_A[y_A == old_class_id] = new_class_id  # reorder class ids for A
    return ((X[~y_for_B], y_A), (X[y_for_B], y_B))


(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

tf.random.set_seed(42)

model_A = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(8, activation="softmax")
])

model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                      validation_data=(X_valid_A, y_valid_A))
model_A.save("my_model_A")

Epoch 1/20
1376/1376 [==============================] - 6s 3ms/step - loss: 1.0987 - accuracy: 0.6467 - val_loss: 0.6847 - val_accuracy: 0.7691
Epoch 2/20
1376/1376 [==============================] - 4s 3ms/step - loss: 0.6033 - accuracy: 0.7994 - val_loss: 0.5222 - val_accuracy: 0.8245
Epoch 3/20
1376/1376 [==============================] - 4s 3ms/step - loss: 0.4917 - accuracy: 0.8429 - val_loss: 0.4489 - val_accuracy: 0.8491
Epoch 4/20
1376/1376 [==============================] - 4s 3ms/step - loss: 0.4316 - accuracy: 0.8615 - val_loss: 0.4049 - val_accuracy: 0.8611
Epoch 5/20
1376/1376 [==============================] - 4s 3ms/step - loss: 0.3926 - accuracy: 0.8731 - val_loss: 0.3749 - val_accuracy: 0.8732
Epoch 6/20
1376/1376 [==============================] - 4s 3ms/step - loss: 0.3649 - accuracy: 0.8803 - val_loss: 0.3534 - val_accuracy: 0.8757
Epoch 7/20
1376/1376 [==============================] - 4s 3ms/step - loss: 0.3452 - accuracy: 0.8847 - val_loss: 0.3380 - val_accuracy:

In [9]:
# train and evaluate model B, without reusing model A

tf.random.set_seed(42)
model_B = tf.keras.Sequential([
    tf.keras.layers.Flatten(input_shape=[28, 28]),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(100, activation="relu",
                          kernel_initializer="he_normal"),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model_B.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.SGD(learning_rate=0.001),
                metrics=["accuracy"])
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))
model_B.evaluate(X_test_B, y_test_B)

Epoch 1/20
7/7 [==============================] - 1s 38ms/step - loss: 0.7114 - accuracy: 0.5400 - val_loss: 0.6988 - val_accuracy: 0.5440
Epoch 2/20
7/7 [==============================] - 0s 22ms/step - loss: 0.6740 - accuracy: 0.5900 - val_loss: 0.6658 - val_accuracy: 0.6231
Epoch 3/20
7/7 [==============================] - 0s 19ms/step - loss: 0.6447 - accuracy: 0.6800 - val_loss: 0.6405 - val_accuracy: 0.6756
Epoch 4/20
7/7 [==============================] - 0s 18ms/step - loss: 0.6185 - accuracy: 0.7300 - val_loss: 0.6170 - val_accuracy: 0.7438
Epoch 5/20
7/7 [==============================] - 0s 18ms/step - loss: 0.5959 - accuracy: 0.7950 - val_loss: 0.5939 - val_accuracy: 0.8160
Epoch 6/20
7/7 [==============================] - 0s 17ms/step - loss: 0.5735 - accuracy: 0.8450 - val_loss: 0.5732 - val_accuracy: 0.8645
Epoch 7/20
7/7 [==============================] - 0s 16ms/step - loss: 0.5527 - accuracy: 0.9050 - val_loss: 0.5536 - val_accuracy: 0.8961
Epoch 8/20
7/7 [===========

[0.4043788015842438, 0.9235000014305115]

In [17]:
model_A = tf.keras.models.load_model('my_model_A')
model_B_on_A = tf.keras.Sequential(model_A.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation='sigmoid'))

Note that model_B_on_A and model_A actually share layers now, so when we train one, it will update both models. If we want to avoid that, we need to build model_B_on_A on top of a clone of model_A:

In [18]:
tf.random.set_seed(42)

In [20]:
model_A_clone = tf.keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [21]:
model_B_on_A = tf.keras.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [22]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])

In [23]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
model_B_on_A.compile(loss="binary_crossentropy", optimizer=optimizer,
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 53ms/step - loss: 0.7796 - accuracy: 0.4400 - val_loss: 0.7439 - val_accuracy: 0.4491
Epoch 2/4
7/7 [==============================] - 0s 25ms/step - loss: 0.7383 - accuracy: 0.4350 - val_loss: 0.7202 - val_accuracy: 0.4510
Epoch 3/4
7/7 [==============================] - 0s 20ms/step - loss: 0.7151 - accuracy: 0.4700 - val_loss: 0.7064 - val_accuracy: 0.5045
Epoch 4/4
7/7 [==============================] - 0s 20ms/step - loss: 0.6902 - accuracy: 0.5200 - val_loss: 0.6872 - val_accuracy: 0.5648
Epoch 1/16
7/7 [==============================] - 1s 46ms/step - loss: 0.6394 - accuracy: 0.6650 - val_loss: 0.5875 - val_accuracy: 0.7280
Epoch 2/16
7/7 [==============================] - 0s 24ms/step - loss: 0.5482 - accuracy: 0.8450 - val_loss: 0.5116 - val_accuracy: 0.8783
Epoch 3/16
7/7 [==============================] - 0s 25ms/step - loss: 0.4778 - accuracy: 0.9050 - val_loss: 0.4624 - val_accuracy: 0.9001
Epoch 4/16
7/7 [===============

In [24]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 3ms/step - loss: 0.2540 - accuracy: 0.9390


[0.25399863719940186, 0.9390000104904175]

### Optimizers

In [25]:
# a little function to test an optimizer on Fashion MNIST

def build_model(seed=42):
    tf.random.set_seed(seed)
    return tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=[28, 28]),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(100, activation="relu",
                              kernel_initializer="he_normal"),
        tf.keras.layers.Dense(10, activation="softmax")
    ])


def build_and_train_model(optimizer):
    model = build_model()
    model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer,
                  metrics=["accuracy"])
    return model.fit(X_train, y_train, epochs=10,
                     validation_data=(X_valid, y_valid))

### SGD

In [29]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001)
history_sgd = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 1.4252 - accuracy: 0.5570 - val_loss: 0.8978 - val_accuracy: 0.7126
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.7930 - accuracy: 0.7341 - val_loss: 0.6897 - val_accuracy: 0.7600
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6643 - accuracy: 0.7730 - val_loss: 0.6031 - val_accuracy: 0.7902
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6002 - accuracy: 0.7951 - val_loss: 0.5558 - val_accuracy: 0.8068
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5598 - accuracy: 0.8070 - val_loss: 0.5297 - val_accuracy: 0.8140
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5317 - accuracy: 0.8170 - val_loss: 0.5140 - val_accuracy: 0.8182
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5117 - accuracy: 0.8231 - val_loss: 0.4893 - val_accuracy:

### Momentum optimization

In [28]:
optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)
history_momentum = build_and_train_model(optimizer) 

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6785 - accuracy: 0.7709 - val_loss: 0.5065 - val_accuracy: 0.8170
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4638 - accuracy: 0.8367 - val_loss: 0.4400 - val_accuracy: 0.8392
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4183 - accuracy: 0.8517 - val_loss: 0.4157 - val_accuracy: 0.8520
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3893 - accuracy: 0.8616 - val_loss: 0.3781 - val_accuracy: 0.8632
Epoch 5/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.3678 - accuracy: 0.8687 - val_loss: 0.3699 - val_accuracy: 0.8686
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3548 - accuracy: 0.8731 - val_loss: 0.3692 - val_accuracy: 0.8656
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3393 - accuracy: 0.8776 - val_loss: 0.3614 - val_accuracy:

### Nesterov Accelerated Gradient

In [30]:

optimizer = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.9,
                                    nesterov=True)
history_nesterov = build_and_train_model(optimizer)

Epoch 1/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6915 - accuracy: 0.7673 - val_loss: 0.4964 - val_accuracy: 0.8226
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4652 - accuracy: 0.8385 - val_loss: 0.4451 - val_accuracy: 0.8400
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4223 - accuracy: 0.8512 - val_loss: 0.4051 - val_accuracy: 0.8548
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3963 - accuracy: 0.8609 - val_loss: 0.3880 - val_accuracy: 0.8610
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3753 - accuracy: 0.8673 - val_loss: 0.3772 - val_accuracy: 0.8648
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3611 - accuracy: 0.8723 - val_loss: 0.3913 - val_accuracy: 0.8598
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.3460 - accuracy: 0.8769 - val_loss: 0.3812 - val_accuracy: